In [1]:
!pip install mlflow textstat

In [2]:
!pip install -q transformers datasets evaluate mlflow accelerate bert-score readability-metrics alignscore

In [3]:
pip install huggingface_hub[hf_xet]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import json
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
import evaluate
import mlflow
import torch
from textstat import flesch_kincaid_grade


with open(r"C:\Users\sanch\Documents\proyecto_grado\data\data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

records = []

for _, articles in data.items():
    if isinstance(articles, dict):  # aseguramos que es un diccionario
        for _, content in articles.items():
            if isinstance(content, dict):  # solo seguimos si es un dict
                # Unir abstract si existe
                abstract = ""
                if "abstract" in content and isinstance(content["abstract"], dict):
                    abstract = " ".join(content["abstract"].values())

                # Unir adaptation si existe
                adaptation = ""
                if "adaptations" in content and isinstance(content["adaptations"], dict):
                    adaptation_dict = content["adaptations"].get("adaptation2", {})
                    if isinstance(adaptation_dict, dict):
                        adaptation = " ".join(adaptation_dict.values())

                if abstract and adaptation:  # solo guardamos pares válidos
                    records.append({"source": abstract, "target": adaptation})

# Convertimos a DataFrame
import pandas as pd
df = pd.DataFrame(records)
print(df.head())
total_ejemplos = len(df)
print(f"Total de ejemplos: {total_ejemplos}")
dataset = Dataset.from_pandas(df)


small_models = [
    "distilgpt2",
    "gpt2",
    "EleutherAI/gpt-neo-125M",
    "EleutherAI/pythia-70m"
]


def tokenize_function(examples, tokenizer):
    inputs = [f"Simplify: {src}" for src in examples["source"]]
    targets = examples["target"]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = labels
    return model_inputs

def train_model(model_name, dataset):
    print(f"\n🔧 Entrenando {model_name}...")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    tokenized = dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)

    model = AutoModelForCausalLM.from_pretrained(model_name)

    if "pythia" in model_name:
        target_modules = ["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"]
    elif "gpt2" in model_name or "ettin" in model_name:
         target_modules = ["c_attn","q_proj","v_proj"]
    else:
         target_modules = None 

    config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=target_modules,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, config)

    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir=f"./results/{model_name.replace('/','_')}",
        eval_strategy="epoch",
        learning_rate=2e-4,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        num_train_epochs=3,   
        weight_decay=0.01,
        save_total_limit=1,
        fp16=True,
        push_to_hub=False,
        logging_dir="./logs",
        report_to="mlflow",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized.shuffle(seed=42).select(range(total_ejemplos)),
        eval_dataset=tokenized.shuffle(seed=42).select(range(100)),
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    trainer.train()
    return model, tokenizer


trained_models = {}
for m in small_models:
    model, tokenizer = train_model(m, dataset)
    trained_models[m] = (model, tokenizer)


bertscore = evaluate.load("bertscore")

def evaluate_model(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt")
    inputs = inputs.to(model.device) 
    outputs = model.generate(**inputs, max_new_tokens=100)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)


    refs = [df["target"][0]]
    preds = [summary]
    bs = bertscore.compute(predictions=preds, references=refs, lang="en")


    fk = flesch_kincaid_grade(summary)

    return summary, bs["f1"][0], fk

sample_text = df["source"][0]
for m, (model, tok) in trained_models.items():
    summary, bert, fk = evaluate_model(model, tok, sample_text)
    print(f"\n Modelo: {m}")
    print(f" Resumen: {summary}")
    print(f" BERTScore F1: {bert:.3f} | Flesch-Kincaid: {fk:.2f}")

                                              source  \
0  Muscle cramps are a common problem characteriz...   
1  The dystonias are a group of disorders charact...   
2  Muscle cramps result in continuous, involuntar...   
3  Exercise-Associated Muscle Cramps (EAMC) are a...   
4  Muscular cramp is a common symptom in healthy ...   

                                              target  
0  Muscle cramps are a common problem represented...  
1  Dystonias are disorders with a lot of uncontro...  
2  Muscle cramps cause constant and unintended co...  
3  Exercise-Associated Muscle Cramps (EAMC) are a...  
4  Muscle cramps are common in healthy people, es...  
Total de ejemplos: 320

🔧 Entrenando distilgpt2...


Map: 100%|██████████| 320/320 [00:00<00:00, 1379.63 examples/s]
c:\Users\sanch\Documents\proyecto_grado\modelos\transalate\venv\Lib\site-packages\peft\tuners\lora\layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
                                                 
 33%|███▎      | 160/480 [04:13<07:21,  1.38s/it]

{'eval_loss': 3.4489240646362305, 'eval_runtime': 31.1532, 'eval_samples_per_second': 3.21, 'eval_steps_per_second': 1.605, 'epoch': 1.0}


                                                 
 67%|██████▋   | 320/480 [07:57<02:59,  1.12s/it]

{'eval_loss': 3.4321718215942383, 'eval_runtime': 24.4517, 'eval_samples_per_second': 4.09, 'eval_steps_per_second': 2.045, 'epoch': 2.0}


...


 Modelo: distilgpt2
 Resumen: Muscle cramps are a common problem characterized by a sudden, painful, involuntary contraction of muscle. These true cramps, which originate from peripheral nerves, may be distinguished from other muscle pain or spasm. Medical history, physical examination, and a limited laboratory screen help to determine the various causes of muscle cramps. Despite the "benign" nature of cramps, many patients find the symptom very uncomfortable. Treatment options are guided both by experience and by a limited number of therapeutic trials. Quinine sulfate is an effective medication, but the side-effect profile is worrisome, and other membrane-stabilizing drugs are probably just as effective. Patients will benefit from further studies to better define the pathophysiology of muscle cramps and to find more effective medications with fewer side-effects.




































































































 BERTScore F1: 0.934 | 

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



 Modelo: gpt2
 Resumen: Muscle cramps are a common problem characterized by a sudden, painful, involuntary contraction of muscle. These true cramps, which originate from peripheral nerves, may be distinguished from other muscle pain or spasm. Medical history, physical examination, and a limited laboratory screen help to determine the various causes of muscle cramps. Despite the "benign" nature of cramps, many patients find the symptom very uncomfortable. Treatment options are guided both by experience and by a limited number of therapeutic trials. Quinine sulfate is an effective medication, but the side-effect profile is worrisome, and other membrane-stabilizing drugs are probably just as effective. Patients will benefit from further studies to better define the pathophysiology of muscle cramps and to find more effective medications with fewer side-effects.

The most common cause of muscle cramps is a sudden, painful, involuntary contraction of muscle. These true cramps, which origina

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



 Modelo: EleutherAI/gpt-neo-125M
 Resumen: Muscle cramps are a common problem characterized by a sudden, painful, involuntary contraction of muscle. These true cramps, which originate from peripheral nerves, may be distinguished from other muscle pain or spasm. Medical history, physical examination, and a limited laboratory screen help to determine the various causes of muscle cramps. Despite the "benign" nature of cramps, many patients find the symptom very uncomfortable. Treatment options are guided both by experience and by a limited number of therapeutic trials. Quinine sulfate is an effective medication, but the side-effect profile is worrisome, and other membrane-stabilizing drugs are probably just as effective. Patients will benefit from further studies to better define the pathophysiology of muscle cramps and to find more effective medications with fewer side-effects.
 BERTScore F1: 0.934 | Flesch-Kincaid: 13.60

 Modelo: EleutherAI/pythia-70m
 Resumen: Muscle cramps are a com